In the previous section, we have explored multiple ways to address the class unbalanceness (average predictions of multiple models, boosting, etc.). In this part, we'll try another popular method that assigns different weights to the terms in the **loss function for different class** (depending on majority or minority class)

Again we'll train simple logistic regression, with **softmax function** (multinomial) rather than sigmoid function. We'll use tensorflow to control the weighting of different terms in the loss function.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [2]:
df = pd.read_csv("creditcard.csv", header=0, sep=",")
df.shape

(284807, 31)

In [3]:
df.columns

Index([u'Time', u'V1', u'V2', u'V3', u'V4', u'V5', u'V6', u'V7', u'V8', u'V9',
       u'V10', u'V11', u'V12', u'V13', u'V14', u'V15', u'V16', u'V17', u'V18',
       u'V19', u'V20', u'V21', u'V22', u'V23', u'V24', u'V25', u'V26', u'V27',
       u'V28', u'Amount', u'Class'],
      dtype='object')

In [4]:
target = df["Class"].values
data = df.ix[:, :-1].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=123, stratify=target)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = OneHotEncoder().fit_transform(y_train[:, np.newaxis]).toarray()

In [6]:
X = tf.placeholder(tf.float32, [None, 30]) 
W = tf.Variable(tf.zeros([30, 2]))
b = tf.Variable(tf.zeros([2]))
y = tf.matmul(X, W) + b

y_ = tf.placeholder(tf.float32, [None, 2])

In [7]:
X.shape

TensorShape([Dimension(None), Dimension(30)])

In [8]:
y.shape

TensorShape([Dimension(None), Dimension(2)])

In [9]:
y_.shape

TensorShape([Dimension(None), Dimension(2)])

In [10]:
ratio = y_train[:, 1].sum() / len(y_train)
class_weight = tf.constant(np.array([ratio, 1 - ratio]) * 10, dtype=tf.float32)

y_weighted = tf.multiply(y, class_weight)
y_weighted.shape

TensorShape([Dimension(None), Dimension(2)])

In [11]:
sess = tf.InteractiveSession()
Epochs = 50
learning_rate = 0.5
splits = 10 # Split training set into `splits` for training in each epoch

for logits, name in zip((y, y_weighted), ("Unweighted", "Weighted")):
    print "Loss function:", name
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    tf.global_variables_initializer().run()
    
    for _ in range(Epochs):
        index_list = np.array_split(range(len(X_train)), splits)
        for index in index_list:
            sess.run(train_step, feed_dict={X: X_train[index, :], y_: y_train[index, :]})
            
    y_pred = sess.run(y, feed_dict={X: X_test})
    y_pred = np.where(y_pred[:, 1] > y_pred[:, 0], 1, 0)            
    
    cm = pd.DataFrame(confusion_matrix(y_test, y_pred))
    cm.index = ["T", "F"]
    cm.index.name = "Target"
    cm.columns = ["T", "F"]
    cm.columns.name = "Predicted"
    
    print "Confusion matrix:"
    print cm
    print "Precision: %f" % precision_score(y_test, y_pred)
    print "Recall: % f" % recall_score(y_test, y_pred)
    print 

Loss function: Unweighted
Confusion matrix:
Predicted      T   F
Target              
T          56855   9
F             44  54
Precision: 0.857143
Recall:  0.551020

Loss function: Weighted
Confusion matrix:
Predicted      T   F
Target              
T          56847  17
F             34  64
Precision: 0.790123
Recall:  0.653061



In [27]:
X = tf.placeholder(tf.float32, [None, 30]) 
W_0 = tf.Variable(tf.zeros([30, 8]))
b_0 = tf.Variable(tf.zeros([8]))
O_0 = tf.nn.sigmoid(tf.matmul(X, W_0) + b_0)

W_1 = tf.Variable(tf.zeros([8, 2]))
b_1 = tf.Variable(tf.zeros([2]))
y = tf.matmul(O_0, W_1) + b_1

y_ = tf.placeholder(tf.float32, [None, 2])

In [28]:
ratio = y_train[:, 1].sum() / len(y_train)
class_weight = tf.constant(np.array([ratio, 1 - ratio]) * 10, dtype=tf.float32)

y_weighted = tf.multiply(y, class_weight)
y_weighted.shape

TensorShape([Dimension(None), Dimension(2)])

In [29]:
sess = tf.InteractiveSession()
Epochs = 50
learning_rate = 0.5
splits = 10 # Split training set into `splits` for training in each epoch

In [30]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_weighted))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
tf.global_variables_initializer().run()
    
for _ in range(Epochs):
    index_list = np.array_split(range(len(X_train)), splits)
    for index in index_list:
        sess.run(train_step, feed_dict={X: X_train[index, :], y_: y_train[index, :]})
            
y_pred = sess.run(y, feed_dict={X: X_test})
y_pred = np.where(y_pred[:, 1] > y_pred[:, 0], 1, 0)            
    
cm = pd.DataFrame(confusion_matrix(y_test, y_pred))

In [31]:
cm

,0,1
0,56837,27
1,25,73


In [32]:
precision_score(y_test, y_pred)

0.72999999999999998

In [33]:
recall_score(y_test, y_pred)

0.74489795918367352